In [1]:
from __future__ import absolute_import, division, print_function

In [2]:
import codecs
import glob
import logging
import multiprocessing
import os
import pprint
import re

import nltk

In [3]:
import tensorflow.keras  as keras

/home/chantana/.local/lib/python2.7/site-packages/h5py/__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [4]:
from nltk import ne_chunk, pos_tag
from nltk.tokenize import sent_tokenize, word_tokenize, PunktSentenceTokenizer
from nltk.corpus import stopwords
import gensim.models.word2vec as w2v
from gensim import logging
import sklearn.manifold
import numpy as np
import matplotlib.pyplot as plt
import pandas as  pd
import seaborn as sns
import pickle
from  keras.preprocessing.text import Tokenizer
from  keras.preprocessing.sequence import pad_sequences
import json
from nltk.tokenize import WhitespaceTokenizer
from nltk.tokenize import MWETokenizer

Using TensorFlow backend.


In [5]:
%pylab inline

Populating the interactive namespace from numpy and matplotlib


In [6]:
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)

In [7]:
import nltk
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /home/chantana/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


True

In [8]:
word_files = ['locationreview.txt']
word_dishes = []
for word_file in word_files:
    with open(word_file) as f:
        lines = f.readlines()
    word_list2 = [x.strip() for x in lines] 
    
    word_dishes.extend(word_list2)
word_dishes[0:5]

['nice place', 'place', 'location', 'great place', 'located']

In [9]:
word_files = ['service.txt']
word_restaurant = []
for word_file in word_files:
    with open(word_file) as f:
        lines = f.readlines()
    word_list2 = [x.strip() for x in lines] 
    
    word_restaurant.extend(word_list2)
word_restaurant[0:5]

['service', 'convenient', 'comfortably', 'be comfortable', 'comfortable']

In [10]:
#remove duplicates
def ordered_set(in_list):
    out_list = []
    added = set()
    for val in in_list:
        if not val in added:
            out_list.append(val)
            added.add(val)
    return out_list

In [11]:
word_dishes = ordered_set(word_dishes)

In [12]:
word_restaurant = ordered_set(word_restaurant)

In [13]:
#word_location = ordered_set(word_location)

In [14]:
mw_list = []
for x in word_dishes:
    ws = x.split() 
    if len(ws) > 1:
        tws = tuple(ws)
        mw_list.append(tws)
        

for x in word_restaurant:
    ws = x.split() 
    if len(ws) > 1:
        tws = tuple(ws)
        mw_list.append(tws)
        

        
#for x in word_location:
#    ws = x.split() 
#    if len(ws) > 1:
#        tws = tuple(ws)
#        mw_list.append(tws)
        
mw_list = ordered_set(mw_list)
mw_list[0:5]

[('nice', 'place'),
 ('great', 'place'),
 ('close', 'to'),
 ('closed', 'to'),
 ('next', 'to')]

In [15]:
my_idx = {}
for i,w in enumerate(mw_list):
    my_idx[" ".join(w)] = i
print(len(my_idx))

62


In [16]:
for i in mw_list:
    if i == ('friendly','service'):
        print('found')
    if i == ('asian','food'):
        print('found')

In [17]:
mwe = MWETokenizer(mw_list,separator=' ')

In [18]:
reviews = pd.read_csv("BERT-hotels-NER.csv")

In [19]:
reviews.shape

(8976, 1)

In [20]:
reviews.head()

,summary
0,00am we would have left immediately we saw the...
1,1 kilometre from patong beach big n clean room...
2,1 min to the main road with all the shops but ...
3,1 min walk max
4,1 score just for near the bts station


In [21]:
reviews = reviews.dropna()
reviews = reviews.reset_index(drop=True)

In [22]:
def spans(txt):
    tokens=mwe.tokenize(word_tokenize(txt.lower()))
    offset = 0
    for token in tokens:
        offset = txt.find(token, offset)
        yield token, offset, offset+len(token)
        offset += len(token)

In [23]:
# A list of contractions from http://stackoverflow.com/questions/19790188/expanding-english-language-contractions-in-python
contractions = { 
"ain't": "am not",
"aren't": "are not",
"can't": "cannot",
"can't've": "cannot have",
"'cause": "because",
"could've": "could have",
"couldn't": "could not",
"couldn't've": "could not have",
"didn't": "did not",
"doesn't": "does not",
"don't": "do not",
"hadn't": "had not",
"hadn't've": "had not have",
"hasn't": "has not",
"haven't": "have not",
"he'd": "he would",
"he'd've": "he would have",
"he'll": "he will",
"he's": "he is",
"how'd": "how did",
"how'll": "how will",
"how's": "how is",
"i'd": "i would",
"i'll": "i will",
"i'm": "i am",
"i've": "i have",
"isn't": "is not",
"it'd": "it would",
"it'll": "it will",
"it's": "it is",
"let's": "let us",
"ma'am": "madam",
"mayn't": "may not",
"might've": "might have",
"mightn't": "might not",
"must've": "must have",
"mustn't": "must not",
"needn't": "need not",
"oughtn't": "ought not",
"shan't": "shall not",
"sha'n't": "shall not",
"she'd": "she would",
"she'll": "she will",
"she's": "she is",
"should've": "should have",
"shouldn't": "should not",
"that'd": "that would",
"that's": "that is",
"there'd": "there had",
"there's": "there is",
"they'd": "they would",
"they'll": "they will",
"they're": "they are",
"they've": "they have",
"wasn't": "was not",
"we'd": "we would",
"we'll": "we will",
"we're": "we are",
"we've": "we have",
"weren't": "were not",
"what'll": "what will",
"what're": "what are",
"what's": "what is",
"what've": "what have",
"where'd": "where did",
"where's": "where is",
"who'll": "who will",
"who's": "who is",
"won't": "will not",
"wouldn't": "would not",
"you'd": "you would",
"you'll": "you will",
"you're": "you are"
}

In [24]:
def clean_text(text, remove_stopwords = True):
    '''Remove unwanted characters, stopwords, and format the text to create fewer nulls word embeddings'''
    
    # Convert words to lower case
    text = text.lower()
    
    # Replace contractions with their longer forms 
    if True:
        # We are not using "text.split()" here
        #since it is not fool proof, e.g. words followed by punctuations "Are you kidding?I think you aren't."
        text = re.findall(r"[\w']+", text)
        new_text = []
        for word in text:
            if word in contractions:
                new_text.append(contractions[word])
            else:
                new_text.append(word)
        text = " ".join(new_text)
    
    # Format words and remove unwanted characters
    text = re.sub(r'https?:\/\/.*[\r\n]*', '', text, flags=re.MULTILINE)# remove links
    text = re.sub(r'\<a href', ' ', text)# remove html link tag
    text = re.sub(r'&amp;', '', text) 
    text = re.sub(r'[_"\-;%()|+&=*%.!?:#$\[\]/]', ' ', text)
    text = re.sub(r'<br />', ' ', text)
    text = re.sub(r'\'', ' ', text)
    
    # Optionally, remove stop words
    if remove_stopwords:
        text = text.split()
        stops = set(stopwords.words("english"))
        text = [w for w in text if not w in stops]
        text = " ".join(text)

    return text

In [25]:
clean_restaurant = []
for restaurants in reviews.summary:
    clean_restaurant.append(clean_text(restaurants, remove_stopwords=False))
print("restaurants are complete.")

restaurants are complete.


In [26]:
from tqdm import tqdm_notebook as tqdm
all_item = []

for i in tqdm(range(len(reviews))):
    word_ls = []
    #print("sentence:",i+1)
    
    for token in spans(clean_restaurant[i]):
        #print(token)
        #assert token[0]==reviews.location[i][token[1]:token[2]]
        my_tuple = token[0]
        #print("###",token)
        
        #my_tuples = ' , '.join(map(str, my_tuple))
        if token[0] in word_dishes:
            #word_ls.append(my_tuple)
            subwords = token[0].split()
            pos_list = [nltk.pos_tag([w]) for w in subwords]
            tag_list = ['I-RLOC']*len(pos_list)
            tag_list[0] = 'B-RLOC' 
            
            
            for s,p,t in zip(subwords,pos_list,tag_list):           
                if type(p) == list:
                    p = p[0][1]
                    #print('list')
                new_item = dict({'Sentence #': i+1, 'Tag' : t, 'Word': s,'POS': p})
                all_item.append(new_item)
                
                
            #lis_lo = nltk.pos_tag(word_ls),LOC
            #print(' , '.join(map(str, lis_lo)))
 
        elif token[0] in word_restaurant:
            
            subwords = token[0].split()
            pos_list = [nltk.pos_tag([w]) for w in subwords]
            tag_list = ['I-RSER']*len(pos_list)
            tag_list[0] = 'B-RSER' 
            
            #print(subwords)
            for s,p,t in zip(subwords,pos_list,tag_list):           
                new_item = dict({'Sentence #': i+1, 'Tag' : t, 'Word': s,'POS': p[0][1]})
                all_item.append(new_item)
                #print(new_item)
            #word_ls.append(my_tuple)
            #print("found LOC")
            #lis_lo = nltk.pos_tag(word_ls),"LOC"
            #print(' , '.join(map(str, lis_lo)))
             
#        elif token[0] in word_location:
            #word_ls.append(my_tuple)
            #lis_lo = nltk.pos_tag(word_ls),"FACILITY"
            #print(' , '.join(map(str, lis_lo)))
            
            #my_pos = nltk.pos_tag([my_tuple])[0][1]
            #new_item =  dict({'Sentence #': i+1, 'Tag' : 'FACILITY', 'Word': my_tuple,'POS': my_pos})
            
#            subwords = token[0].split()
#            pos_list = [nltk.pos_tag([w]) for w in subwords]
#            tag_list = ['I-SHOP']*len(pos_list)
#            tag_list[0] = 'B-SHOP' 
            
             
#            for s,p,t in zip(subwords,pos_list,tag_list):   
#                if type(p) == list:
#                    p = p[0][1]
#                new_item = dict({'Sentence #': i+1, 'Tag' : t, 'Word': s,'POS': p})
#                all_item.append(new_item)
                

             
        else:
            #print(type(my_tuple))
            my_pos = nltk.pos_tag([my_tuple])[0][1]
            new_item = dict({'Sentence #': i+1, 'Tag' : 'O', 'Word': my_tuple.lower(),'POS': my_pos})
            all_item.append(new_item)
            
            #if i+1 == 5177:
                #print(new_item)
             
            #print(nltk.pos_tag([my_tuple]),',','O')
        #print(new_item)
        #if not(my_pos == '.' or  my_pos == ',' or my_pos == ':' or my_pos == '(' or my_pos == ')') :
            
            #all_item.append(new_item) 

file_csv = 'bert-hotels.csv'
with open(file_csv, 'w') as csv_file:
    csv_file.write('Sentence #*Word*POS*Tag\n')
    for item in all_item:      
        #print(item['POS'])    
        csv_file.write(str(item['Sentence #'])+'*'+item['Word']+'*'+item['POS']+'*'+item['Tag']+'\n')
csv_file.close()

In [27]:
import pandas as pd
import pandas as pd
import numpy as np
from tqdm import tqdm, trange
#load data
# explore data
file_csv = 'bert-hotels.csv'
data = pd.read_csv(file_csv, sep='*',encoding="utf-8").fillna(method="ffill")

data.head(10)

,Sentence #,Word,POS,Tag
0,1,00am,CD,O
1,1,we,PRP,O
2,1,would,MD,O
3,1,have,VB,O
4,1,left,NN,O
5,1,immediately,RB,O
6,1,we,PRP,O
7,1,saw,NN,O
8,1,the,DT,O
9,1,bedrooms,NNS,B-RSER


In [28]:
class SentenceGetter(object):
    
    def __init__(self, data):
        self.n_sent = 1
        self.data = data
        self.empty = False
        agg_func = lambda s: [(w, p, t) for w, p, t in zip(s["Word"].values.tolist(),
                                                           s["POS"].values.tolist(),
                                                           s["Tag"].values.tolist())]
        self.grouped = self.data.groupby("Sentence #").apply(agg_func)
        self.sentences = [s for s in self.grouped]
    
    def get_next(self):
        try:
            s = self.grouped["Sentence: {}".format(self.n_sent)]
            self.n_sent += 1
            return s
        except:
            return None

In [29]:
#concat sentence
getter = SentenceGetter(data)

In [30]:
word_list = [ [s[0] for s in sent] for sent in getter.sentences] 

In [31]:
sentences = word_list
sentences[0]

[u'00am',
 u'we',
 u'would',
 u'have',
 u'left',
 u'immediately',
 u'we',
 u'saw',
 u'the',
 u'bedrooms']

In [32]:
print(len(sentences))

8976


In [33]:
labels = [[s[2] for s in sent] for sent in getter.sentences]
print(labels[0])
print(len(labels[0]))

[u'O', u'O', u'O', u'O', u'O', u'O', u'O', u'O', u'O', u'B-RSER']
10


In [34]:
print(len(word_list[0]))

10


In [35]:
print(len(labels))

8976


In [36]:
tags_vals = list(set(data["Tag"].values))
tag2idx = {t: i for i, t in enumerate(tags_vals)}
idx2tag = {i: t for i, t in enumerate(tags_vals) }

In [37]:
words = list(set(data["Word"].values))
n_words = len(words); 
word2idx = {w: i + 2 for i, w in enumerate(words)}
word2idx["UNK"] = 1
word2idx["PAD"] = 0
idx2word = {i: w for w, i in word2idx.items()}
n_words

4963

In [38]:
idx2tag[0]

u'I-RSER'

In [39]:
import sys
sys.path.append('pytorch-pretrained-BERT/')

In [40]:
import torch
from torch.optim import Adam
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
from keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split
from pytorch_pretrained_bert import BertTokenizer, BertConfig
from pytorch_pretrained_bert import BertForTokenClassification, BertAdam

Better speed can be achieved with apex installed from https://www.github.com/nvidia/apex.


In [41]:
MAX_LEN = 128
bs = 8
device = torch.device("cuda")

n_gpu = torch.cuda.device_count()
#torch.cuda.get_device_name(0)
torch.cuda.set_device(1)

In [42]:
tokenized_texts = word_list
print(tokenized_texts[0])

[u'00am', u'we', u'would', u'have', u'left', u'immediately', u'we', u'saw', u'the', u'bedrooms']


In [43]:
sentences[0]

[u'00am',
 u'we',
 u'would',
 u'have',
 u'left',
 u'immediately',
 u'we',
 u'saw',
 u'the',
 u'bedrooms']

In [44]:
tokens_ids = [[word2idx[w] for w in s] for s in tokenized_texts]
print(len(tokens_ids[0]),len(labels[0]))

10 10


In [45]:
 print(len(tokens_ids[0]),len(labels[0]))

10 10


In [46]:
#input_ids = pad_sequences([tokenizer.convert_tokens_to_ids(txt) for txt in tokenized_texts],
input_ids = pad_sequences(tokens_ids,
                          maxlen=MAX_LEN, dtype="int64", truncating="post", padding="post")
#input_ids = pad_sequences([convert_word2idx(txt) for txt in tokenized_texts],
                          #maxlen=MAX_LEN, dtype="int64", truncating="post", padding="post")

In [47]:
print(MAX_LEN)
for i in tokens_ids:
    if len(i) > MAX_LEN:
         
        i = i[:128]
        print("need more")
         
        #MAX_LEN = len(i)
print(MAX_LEN)

128
128


In [48]:
t_list = [[tag2idx.get(l) for l in lab] for lab in labels]
len(t_list[0])

10

In [49]:
print(input_ids[0])

[ 405  729  164 4300  918 4716  729 1785  914 3807    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0]


In [50]:
tags = pad_sequences([[tag2idx.get(l) for l in lab] for lab in labels],
                     maxlen=MAX_LEN, value=tag2idx["O"], padding="post",
                     dtype="int64", truncating="post")

In [51]:
attention_masks = [[float(i>0) for i in ii] for ii in input_ids]

In [52]:
#split train test
tr_inputs, val_inputs, tr_tags, val_tags = train_test_split(input_ids, tags, 
                                                            random_state=2018, test_size=0.1)
tr_masks, val_masks, _, _ = train_test_split(attention_masks, input_ids,
                                             random_state=2018, test_size=0.1)

In [53]:
#change to torhc tensor
tr_inputs = torch.tensor(tr_inputs)
val_inputs = torch.tensor(val_inputs)
tr_tags = torch.tensor(tr_tags)
val_tags = torch.tensor(val_tags)
tr_masks = torch.tensor(tr_masks)
val_masks = torch.tensor(val_masks)

In [54]:
print(tr_inputs.shape)
print(tr_masks.shape)
print(tr_tags.shape)

torch.Size([8078, 128])
torch.Size([8078, 128])
torch.Size([8078, 128])


In [55]:
print(val_inputs.shape)
print(val_masks.shape)
print(val_tags.shape)

torch.Size([898, 128])
torch.Size([898, 128])
torch.Size([898, 128])


In [56]:
train_data = TensorDataset(tr_inputs, tr_masks, tr_tags)
train_sampler = RandomSampler(train_data)
train_dataloader = DataLoader(train_data, sampler=train_sampler, batch_size=bs)

valid_data = TensorDataset(val_inputs, val_masks, val_tags)
valid_sampler = SequentialSampler(valid_data)
valid_dataloader = DataLoader(valid_data, sampler=valid_sampler, batch_size=bs)

In [57]:
print(len(tag2idx))

5


In [58]:
model = BertForTokenClassification.from_pretrained(u"bert-large-uncased", num_labels=len(tag2idx))

2019-10-31 07:54:13,287 : INFO : loading archive file https://s3.amazonaws.com/models.huggingface.co/bert/bert-large-uncased.tar.gz from cache at /home/chantana/.pytorch_pretrained_bert/214d4777e8e3eb234563136cd3a49f6bc34131de836848454373fa43f10adc5e.abfbb80ee795a608acbf35c7bf2d2d58574df3887cdd94b355fc67e03fddba05
2019-10-31 07:54:13,288 : INFO : extracting archive file /home/chantana/.pytorch_pretrained_bert/214d4777e8e3eb234563136cd3a49f6bc34131de836848454373fa43f10adc5e.abfbb80ee795a608acbf35c7bf2d2d58574df3887cdd94b355fc67e03fddba05 to temp dir /tmp/tmpBL09DY
2019-10-31 07:54:23,813 : INFO : Model config {
  "attention_probs_dropout_prob": 0.1, 
  "hidden_act": "gelu", 
  "hidden_dropout_prob": 0.1, 
  "hidden_size": 1024, 
  "initializer_range": 0.02, 
  "intermediate_size": 4096, 
  "max_position_embeddings": 512, 
  "num_attention_heads": 16, 
  "num_hidden_layers": 24, 
  "type_vocab_size": 2, 
  "vocab_size": 30522
}

2019-10-31 07:54:34,655 : INFO : Weights of BertForTokenCla

In [59]:
model.cuda();

In [60]:
FULL_FINETUNING = True
if FULL_FINETUNING:
    param_optimizer = list(model.named_parameters())
    no_decay = ['bias', 'gamma', 'beta']
    optimizer_grouped_parameters = [
        {'params': [p for n, p in param_optimizer if not any(nd in n for nd in no_decay)],
         'weight_decay_rate': 0.01},
        {'params': [p for n, p in param_optimizer if any(nd in n for nd in no_decay)],
         'weight_decay_rate': 0.0}
    ]
else:
    param_optimizer = list(model.classifier.named_parameters()) 
    optimizer_grouped_parameters = [{"params": [p for n, p in param_optimizer]}]
optimizer = Adam(optimizer_grouped_parameters, lr=3e-5)

In [61]:
#Fine tune BERT
from seqeval.metrics import f1_score

def flat_accuracy(preds, labels):
    pred_flat = np.argmax(preds, axis=2).flatten()
    labels_flat = labels.flatten()
    return np.sum(pred_flat == labels_flat) / len(labels_flat)

In [62]:
torch.cuda.set_device(1)

In [ ]:
epochs = 10
max_grad_norm = 1.0

for _ in trange(epochs, desc="Epoch"):
    # TRAIN loop
    model.train()
    tr_loss = 0
    nb_tr_examples, nb_tr_steps = 0, 0
    for step, batch in enumerate(train_dataloader):
        # add batch to gpu
        batch = tuple(t.to(device) for t in batch)
        b_input_ids, b_input_mask, b_labels = batch
        # forward pass
        loss = model(b_input_ids, token_type_ids=None,
                     attention_mask=b_input_mask, labels=b_labels)
        # backward pass
        loss.backward()
        # track train loss
        tr_loss += loss.item()
        nb_tr_examples += b_input_ids.size(0)
        nb_tr_steps += 1
        # gradient clipping
        torch.nn.utils.clip_grad_norm_(parameters=model.parameters(), max_norm=max_grad_norm)
        # update parameters
        optimizer.step()
        model.zero_grad()
    # print train loss per epoch
    print("Train loss: {}".format(tr_loss/nb_tr_steps))
    # VALIDATION on validation set
    model.eval()
    eval_loss, eval_accuracy = 0, 0
    nb_eval_steps, nb_eval_examples = 0, 0
    predictions , true_labels, true_inputs = [], [],[]
    for batch in valid_dataloader:
        batch = tuple(t.to(device) for t in batch)
        b_input_ids, b_input_mask, b_labels = batch
        
        with torch.no_grad():
            tmp_eval_loss = model(b_input_ids, token_type_ids=None,
                                  attention_mask=b_input_mask, labels=b_labels)
            logits = model(b_input_ids, token_type_ids=None,
                           attention_mask=b_input_mask)
        logits = logits.detach().cpu().numpy()
        label_ids = b_labels.to('cpu').numpy()
        inputs = b_input_ids.to('cpu').numpy()
         
        true_inputs.append(inputs)
        predictions.extend([list(p) for p in np.argmax(logits, axis=2)])
        true_labels.append(label_ids)
        
        tmp_eval_accuracy = flat_accuracy(logits, label_ids)
        
        eval_loss += tmp_eval_loss.mean().item()
        eval_accuracy += tmp_eval_accuracy
        
        nb_eval_examples += b_input_ids.size(0)
        nb_eval_steps += 1
    eval_loss = eval_loss/nb_eval_steps
    print("Validation loss: {}".format(eval_loss))
    print("Validation Accuracy: {}".format(eval_accuracy/nb_eval_steps))
    pred_tags = [tags_vals[p_i] for p in predictions for p_i in p]
    valid_tags = [tags_vals[l_ii] for l in true_labels for l_i in l for l_ii in l_i]
    valid_inputs = [[idx2word[l_ii] for l_ii in l_i] for l in  true_inputs  for l_i in l ]


    print("F1-Score: {}".format(f1_score(pred_tags, valid_tags)))

Epoch:   0%|          | 0/10 [00:00<?, ?it/s]

Train loss: 0.330112107068
Validation loss: 0.158912696637
Validation Accuracy: 0.926455337389


Epoch:  10%|█         | 1/10 [07:23<1:06:30, 443.40s/it]

F1-Score: 0.187523918867
Train loss: 0.116663709016
Validation loss: 0.0718912985003
Validation Accuracy: 0.959451050885


Epoch:  20%|██        | 2/10 [14:54<59:25, 445.66s/it]  

...

F1-Score: 0.519736842105
Train loss: 0.0365385260629
Validation loss: 0.0352332730658
Validation Accuracy: 0.99040721792


Epoch:  70%|███████   | 7/10 [52:35<22:32, 450.96s/it]

F1-Score: 0.717163120567
Train loss: 0.0392039170039
Validation loss: 0.0449617129268
Validation Accuracy: 0.991020810288


Epoch:  80%|████████  | 8/10 [1:00:07<15:02, 451.43s/it]

F1-Score: 0.709677419355


In [ ]:
count = 0 
all_data = 0
preds = []
vals = []
for i,j,ll in zip(pred_tags,valid_tags,val_inputs):
    for k,l,kk in zip(i,j,ll):
        preds.append(k)
        vals.append(l)
        if k==l   : 
            count += 1
            #print(k,l,idx2word[kk.item()])
        all_data += 1
print(count)
print(all_data)
from sklearn.metrics import classification_report
target_names = [u'B-RSER', u'I-RSER',u'B-RLOC',u'I-RLOC','0']
print(classification_report(preds,vals, target_names=target_names))